In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#installs
!pip install azure-storage-blob
!pip install boto3
!pip install google-cloud-storage
!pip install pyarrow

In [6]:
#import libraries

import pandas as pd
import numpy as np
import json
import requests
import boto3
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from google.cloud import storage
from io import StringIO

In [14]:
# Azure Functions
def azure_upload_blob(connect_str, container_name, blob_name, data):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True)
    print(f"Uploaded to Azure Blob: {blob_name}")

def azure_download_blob(connect_str, container_name, blob_name):
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    download_stream = blob_client.download_blob()
    return download_stream.readall()

In [ ]:
#read file
df_raw = pd.read_excel(r'/content/gdrive/My Drive/Queens Rolling Sales 2022/Queens_Rolling_2022.xlsx')
df_raw.head()
print(df_raw.columns)

In [ ]:
#display data
display(pd.DataFrame(df_raw))

In [ ]:
#data types
df_raw.info()

In [12]:
df_cleaned = df_raw.copy()

In [ ]:
df_cleaned = df_raw.drop(columns = ['EASE-MENT'])
df_cleaned = df_raw[df_raw['PRICE PER SQUARE FOOT'] != 0]
df_cleaned = df_raw[df_raw['PRICE PER SQUARE FOOT'] != '#DIV/0!']
#display(pd.DataFrame(df_cleaned))
df_cleaned.info()

In [31]:
#specify the path to your JSON configuration file
config_file_path = "config.json"

#load with JSON configuration file
with open(config_file_path, 'r') as config_file:
    config = json.load(config_file)

CONNECTION_STRING_AZURE_STORAGE = config["connectionString"]
CONTAINER_AZURE = 'queensrollingsales'
blob_name = "queensrollingsales_20240402.csv"

#covert Dataframe to csv
output = StringIO()
df_raw.to_csv(output, index=False)
data = output.getvalue()
output.close()

#create the blobservice client object
blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING_AZURE_STORAGE)

#get a blob client using the container and blob name
blob_client = blob_service_client.get_blob_client(container=CONTAINER_AZURE,blob=blob_name)

#upload the csv data to Azure Cloud
blob_client.upload_blob(data, overwrite=True)

print(f"Uploaded {blob_name} to Azure Blob Storage in container {CONTAINER_AZURE}.")

Uploaded queensrollingsales_20240402.csv to Azure Blob Storage in container queensrollingsales.
